# PMR3508 - Aprendizado de Máquina e Reconhecimento de Padrões
                                      Bases Adult e HouseholdIncome - Classificador kNN

* Autor: PMR3508-2019-42

## 1. Base Adult

Imports necessários:

In [ ]:
# Manipulação dos dados
import pandas as pd
import numpy as np

# Visualização 
import matplotlib.pyplot as plt
import os

# Técnica de Machine Learning
import sklearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn import preprocessing as prep

In [ ]:
os.listdir("../input/uci-adult")

### 1.1 Entendendo os dados

In [ ]:
adult = pd.read_csv("../input/uci-adult/adult.data",
        names = [
            "Age", "Workclass", "fnlwgt", "Education", "Education-Num", "Martial Status",
            "Occupation", "Relationship", "Race", "Sex", "Capital Gain", "Capital Loss",
            "Hours per week", "Country", "Target"],
        sep= r'\s*,\s*',
        engine= 'python',
        na_values= "?")

adult.shape

In [ ]:
adult.head()

#### 1.11 Análise das features mais relevantes

In [ ]:
adult.describe()

In [ ]:
adult["Workclass"].value_counts().plot(kind = "bar")

In [ ]:
adult["Martial Status"].value_counts()

In [ ]:
adult["Occupation"].value_counts().plot(kind = 'bar')

In [ ]:
adult["Relationship"].value_counts().plot(kind = 'bar')

In [ ]:
adult["Race"].value_counts().plot(kind = 'pie')

In [ ]:
adult["Sex"].value_counts().plot(kind = 'pie')

In [ ]:
adult["Country"].value_counts()

Conclusões

* Na feature "Country" predomina USA, logo, é descartável

* "Education" descartável, pois há "Education-Num"

* "fnlwgt" será descartado, pois não é relevante para a predição

#### 1.12 Análise dos dados faltantes

In [ ]:
adult.isnull().sum()

As features 'Workclass', 'Occupation' e 'Country' concentram os dados faltantes

* Estratégia: inserir a moda da feature no lugar dos dados faltantes

In [ ]:
moda = adult['Workclass'].describe().top
adult['Workclass'] = adult['Workclass'].fillna(moda)

moda = adult['Occupation'].describe().top
adult['Occupation'] = adult['Occupation'].fillna(moda)

moda = adult['Country'].describe().top
adult['Country'] = adult['Country'].fillna(moda)

adult.isnull().sum()

#### 1.13 Dados de Teste

In [ ]:
testAdult = pd.read_csv("../input/uci-adult/adult.test",
            names = [
            "Age", "Workclass", "fnlwgt", "Education", "Education-Num", "Martial Status",
            "Occupation", "Relationship", "Race", "Sex", "Capital Gain", "Capital Loss",
            "Hours per week", "Country", "Target"],
        sep= r'\s*,\s*',
        engine= 'python',
        na_values= "?")

testAdult.shape

In [ ]:
testAdult.isnull().sum()

A estratégia de tratamento de dados faltantes será a mesma que foi aplicada aos dados de treino.
Como a feature "fnlwgt" não é relevante, somente a feature "Relationship" será tratada. Em "Target" a exclusão das linhas é mais conveniente.

In [ ]:
moda = testAdult['Workclass'].describe().top
testAdult['Workclass'] = testAdult['Workclass'].fillna(moda)

moda = testAdult['Occupation'].describe().top
testAdult['Occupation'] = testAdult['Occupation'].fillna(moda)

moda = testAdult['Country'].describe().top
testAdult['Country'] = testAdult['Country'].fillna(moda)

testAdult.isnull().sum()

In [ ]:
nTestAdult = testAdult.dropna()

nTestAdult.shape

In [ ]:
nTestAdult.head()

#### 1.14 Transformação dos dados não-numéricos em valores numéricos

In [ ]:
adult = adult.apply(prep.LabelEncoder().fit_transform)

nTestAdult = nTestAdult.apply(prep.LabelEncoder().fit_transform)

In [ ]:
adult.head()

### 1.2 Classificador k-NN

* Primeiro Teste: k = 5 usando somente dados númericos

In [ ]:
atributos = ["Age", "Education-Num", "Capital Gain", "Capital Loss", "Hours per week"]

x_train = adult[atributos]
y_train = adult.Target

x_test = nTestAdult[atributos]
y_test = nTestAdult.Target

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 5)

scores = cross_val_score(knn, x_train, y_train, cv=10)
scores.mean()

In [ ]:
knn.fit(x_train, y_train)

y_predict = knn.predict(x_test)

accuracy_score(y_test, y_predict)

* Segundo Teste: k = 25 usando somente dados númericos

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 25)

scores = cross_val_score(knn, x_train, y_train, cv=10)
scores.mean()

In [ ]:
knn.fit(x_train, y_train)

y_predict = knn.predict(x_test)

accuracy_score(y_test, y_predict)

* Terceiro Teste: k = 25 usando dados númericos e não numéricos

In [ ]:
atributos = ["Age", "Workclass", "Education-Num", "Occupation", "Race", "Capital Gain", "Capital Loss", "Hours per week"]

x_train = adult[atributos]
y_train = adult.Target

x_test = nTestAdult[atributos]
y_test = nTestAdult.Target

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 25)

scores = cross_val_score(knn, x_train, y_train, cv=10)
scores.mean()

In [ ]:
knn.fit(x_train, y_train)

y_predict = knn.predict(x_test)

accuracy_score(y_test, y_predict)

Pelos resultados é perceptível que a mescla de atributos numéricos com não numéricos apresentou melhor resultado. Agora deve-se encontrar o melhor hiperparâmetro k

#### 1.21 Treinamento e Validação Cruzada para encontrar o melhor hiperparâmetro k

In [ ]:
    inf = 1
    sup = 35

    scores_media = []
    aux = 0
    k_max = 0

    i = 0
    for k in range(inf, sup):
        knn = KNeighborsClassifier(n_neighbors = k)
        scores = cross_val_score(knn, x_train, y_train, cv=10)
        scores_media.append(scores.mean())

        if scores_media[i] > aux:
            k_max = k
            aux = scores_media[i]

        i = i + 1

    print(k_max)

In [ ]:
x = np.arange(1, sup)

plt.figure(figsize=(10, 5))
plt.plot(x, scores_media, '--', color = 'red', linewidth = 2)
plt.plot(k_max, scores_media[k_max], 'o')

plt.xlabel('k')
plt.ylabel('Acurácia')
plt.title('Perfomance do algoritmo conforme o valor de k')

In [ ]:
print('Acurácia para k = {0} : {1:2.2f}%'.format(k_max, 100 * scores_media[k_max]))

No gráfico acima pode-se observar a acurácia obtida através da validação cruzada conforme o valor de k.

#### 1.22 Teste com o melhor hiperparâmetro

In [ ]:
k = k_max

knn = KNeighborsClassifier(n_neighbors = k)

In [ ]:
knn.fit(x_train, y_train)

In [ ]:
y_predict = knn.predict(x_test)

y_predict

#### 1.23 Acurácia

In [ ]:
accuracy_score(y_test, y_predict)

#### 1.24 Submissão dos Resultados

In [ ]:
predict = []

for i in range(len(y_predict)):
    if y_predict[i] == 0:
        predict.append('<=50K')
    else:
        predict.append('>50K')

result = pd.DataFrame(predict, columns = ["income"])
result.to_csv("Resultados_Adult.csv", index_label="Id")

result

## 2. Base HouseholdIncome - Exercício Extra

Imports necessários:

In [ ]:
# Manipulação dos dados
import pandas as pd
import numpy as np

# Visualização 
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Técnica de Machine Learning
import sklearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn import preprocessing

### 2.1 - Entendendo os dados

Determinação das bases

In [ ]:
os.listdir("../input/costa-rican-household-poverty-prediction/")

In [ ]:
train = pd.read_csv('../input/costa-rican-household-poverty-prediction/train.csv')
test = pd.read_csv('../input/costa-rican-household-poverty-prediction/test.csv')

train.info()

train.head()

In [ ]:
test.info()

test.head()

Tradução dos rótulos

* 1 = extreme poverty 
* 2 = moderate poverty 
* 3 = vulnerable households 
* 4 = non vulnerable households

Tipos de dados

* Dados Int: certamente se referem às features binárias
* Dados floats: referem-se às variáveis contínuas 
* Dados object: não serão usados devido a dificuldade em trabalhar com machine learning

Análise da distribuição das variáveis contínuas conforme o rótulo

In [ ]:
from collections import OrderedDict

plt.figure(figsize = (20, 16))

cores = OrderedDict({1: 'red', 2: 'orange', 3: 'blue', 4: 'green'})
pobreza = OrderedDict({1: 'extreme', 2: 'moderate', 3: 'vulnerable', 4: 'non vulnerable'})

for i, col in enumerate(train.select_dtypes('float')):
    ax = plt.subplot(4, 2, i + 1)
    
    for nivel, cor in cores.items():
       
        sns.kdeplot(train.loc[train['Target'] == nivel, col].dropna(), 
                    ax = ax, color = cor, label = pobreza[nivel])
        
    plt.title(f'{col.capitalize()} Distribution'); plt.xlabel(f'{col}'); plt.ylabel('Density')
    
    plt.subplots_adjust(top = 2)

Como a feature "parentesco1" indica o chefe da família, que reprenta toda a família e assim está diretamente relacionado ao rótulo, passa a ser interessante saber a distribuição dos rótulos.

In [ ]:
rotulos = train.loc[(train['Target'].notnull()) & (train['parentesco1'] == 1), ['Target', 'idhogar']]

quantidade = rotulos['Target'].value_counts().sort_index()

quantidade.plot.bar(figsize = (8, 6), color = cores.values())

plt.xlabel('Nível de Pobreza')
plt.ylabel('Quantidade')
plt.title('Quantidade para cada Nível de Pobreza')


quantidade

Há mais famílias em situação de não vulnerabilidade (rótulo 4).

Tal distribuição pode representar um problema, já que em média podemos afirmar que a população da Costa Rica não está em vulnerabilidade, o que de fato é verdade pela distribuição, mas tal configuração pode atrapalhar a classificação do modelo de machine learning

#### 2.11 Identificando erros

Devido ao problema de distribuição talvez pessoas da mesma família tenham rótulos diferentes, então é necessário verificar tal situação

In [ ]:
igualdade = train.groupby('idhogar')['Target'].apply(lambda x: x.nunique() == 1)

desigualdade = igualdade[igualdade != True]

print('Há {} famílias com membros de rótulos diferentes'.format(len(desigualdade)))

A partir do resultado é conveniente padronizar a família pelo rótulo do chefe, mas como a acurácia será medida somente em relação ao chefe, então não será necessário

#### 2.12 Limpeza dos dados

In [ ]:
colunas_nulas = train.isnull().sum().sort_values(ascending = False)

porcentagem = ((train.isnull().sum()/train.isnull().count())*100).sort_values(ascending = False)

faltantes = pd.concat([colunas_nulas, porcentagem], axis = 1, keys = ['Total', '%'])
faltantes.head()

* v2a1 - Pagamento mensal de aluguel

Tal atributo se relaciona diretamente com a distribuição de 'tipovivi1' que refere-se a quem tem a casa própria e totalmente paga. A estratégia então é preencher com zero o valor da renda mensal de quem possui a casa totalmente paga

Para os demais dados faltantes a alternativa será retirar os dados

In [ ]:
train.loc[(train['tipovivi1'] == 1), 'v2a1'] = 0

print('Ainda restam {} dados faltantes para v2a1'.format(train['v2a1'].isnull().sum()))

* rez_esc - Anos atrasado na escola

Tal atributo se relaciona com a idade do indivíduo, então é interessante analisar os dados faltantes de acordo com a idade

In [ ]:
train.loc[train['rez_esc'].notnull()]['age'].describe()

In [ ]:
train.loc[train['rez_esc'].isnull()]['age'].describe()

A maioria dos dados faltantes se relacionam com adultos, enquanto os dados presentes se relacionam com os jovens, no caso de no máximo 17 anos.
Na descrição da base é possível verificar que 'rez_esc' é definida apenas para indivíduos entre 7 e 19 anos, logo, é conveniente zerar o atributo dos dados faltantes dos indivíduos fora da faixa de idade.

In [ ]:
train.loc[((train['age'] > 19) | (train['age'] < 7)) & (train['rez_esc'].isnull()), 'rez_esc'] = 0

print('Ainda restam {} dados faltantes para v2a1'.format(train['rez_esc'].isnull().sum()))

* v18q1 - Números de tablets que a família possui

Tal atributo é irrelevante pela intuição, então não será tratado

* meaneduc - média de anos de educação para adultos
* SQBmeaned - quadrado da média de anos de educação dos adultos na casa

Será analisado o boxplot de cada atributo

In [ ]:
train['meaneduc'].plot(kind = 'box', grid = True)

In [ ]:
train['SQBmeaned'].plot(kind = 'box', grid = True)

Analisando os box plots, ambos não apresentam tamanha desigualdade, então considerar os dados faltantes como a média da amostra passa a ser uma estratégia interessante.

In [ ]:
train['meaneduc'] = train['meaneduc'].fillna(train['meaneduc'].describe().mean())

train['SQBmeaned'] = train['SQBmeaned'].fillna(train['SQBmeaned'].describe().mean())

O restante das linhas com dados faltantes serão eliminadas, mas como 'v18q1' não será levado em consideração na classificação é conveniente filtrar a base pelos atributos relevantes antes do tratamento final dos dados faltantes, como forma de reduzir a quantidade de dados perdidos

#### 2.13 Seleção dos atributos

A partir da descrição da base optou-se por selecionar as features mais relevantes por intuição:

* v2a1: pagamento mensal do aluguel
* hacdor: superlotação de quartos
* v14a: presença de banheiro a casa
* escolari: anos de escolaridade
* rez_esc: anos atrasado na escola
* hhsize: tamanho da casa
* cielorazo: presença de teto na casa
* abastaguadentro: abastecimento de água dentro da casa
* abastaguano: presença de abastecimento de aágua
* noelec: sem eletricidade na casa
* sanitario1: sem banheiro na casa
* hogar_nin: número de crianças entre 0 e 19 anos na casa
* hogar_total: número total de indivíduos na casa
* meaneduc: média de anos de educação para adultos
* tipovivi1: casa própria e totalmente paga
* area1: zona urbana
* SQBovercrowding: superlotação ao quadrado
* SQBmeaned: quadrado da média de anos de educação dos adultos na casa

In [ ]:
atributos = ['v2a1', 'hacdor', 'v14a', 'escolari', 'rez_esc', 'hhsize', 'cielorazo', 'abastaguadentro',
             'abastaguano', 'noelec', 'sanitario1', 'hogar_nin', 'hogar_total', 'meaneduc', 'tipovivi1',
             'area1', 'SQBovercrowding', 'SQBmeaned', 'Target']

# Nova base de dados
base = train[atributos]
base = base.astype(np.float)

print(base.shape)
base.head()

In [ ]:
colunas_nulas = base.isnull().sum().sort_values(ascending = False)

porcentagem = ((base.isnull().sum()/base.isnull().count())*100).sort_values(ascending = False)

faltantes = pd.concat([colunas_nulas, porcentagem], axis = 1, keys = ['Total', '%'])
faltantes.head()

In [ ]:
# Retirando os demais dados faltantes

base = base.dropna()
base.shape

In [ ]:
colunas_nulas = base.isnull().sum().sort_values(ascending = False)

porcentagem = ((base.isnull().sum()/base.isnull().count())*100).sort_values(ascending = False)

faltantes = pd.concat([colunas_nulas, porcentagem], axis = 1, keys = ['Total', '%'])
faltantes.head()

### 2.2 - Classificador k-NN


In [ ]:
# Treino

x_train = base.drop('Target', axis = 1)

y_train = base['Target']

#### 2.21 Seleção do hiperparâmetro k por Validação Cruzada

In [ ]:
inf = 1
sup = 65

scores_media = []
aux = 0
k_max = 0

i = 0
for k in range(inf, sup):
    knn = KNeighborsClassifier(n_neighbors = k)
    scores = cross_val_score(knn, x_train, y_train, cv=10)
    scores_media.append(scores.mean())
    
    if scores_media[i] > aux:
        k_max = k
        aux = scores_media[i]
        
    i = i + 1
    
print(k_max)

In [ ]:
x = np.arange(1, sup)

plt.figure(figsize=(10, 5))
plt.plot(x, scores_media, '--', color = 'red', linewidth = 2)
plt.plot(k_max, scores_media[k_max], 'o')

plt.xlabel('k')
plt.ylabel('Acurácia')
plt.title('Perfomance do algoritmo conforme o valor de k')

In [ ]:
print('Acurácia para k = {0} : {1:2.2f}%'.format(k_max, 100 * scores_media[k_max]))

No gráfico acima pode-se observar a acurácia obtida através da validação cruzada conforme o valor de k.